In [7]:
import os
import malaya_speech
from malaya_speech import Pipeline
import gc
import librosa
import numpy as np
import torch
from tqdm import tqdm
import joblib

def norm_mel(y, sr):
    mel = librosa.feature.melspectrogram(y, sr = sr, n_mels = 80,hop_length=512)
    mel = librosa.power_to_db(mel, ref=np.max)
    return mel.T

quantized_model = malaya_speech.vad.deep_model(model = 'vggvox-v2', quantized = True)
p = Pipeline()

pipeline_left = (
    p.map(malaya_speech.generator.frames, frame_duration_ms = 30, sample_rate = 16000)
)

pipeline_right = (
    pipeline_left.batching(5)
    .foreach_map(quantized_model.predict)
    .flatten()
)

pipeline_left.foreach_zip(pipeline_right).map(malaya_speech.combine.without_silent,
                                             threshold_to_stop = 0.05)


def remove_silent(y,sr,time_length=80000):
    y =  p(y)['without_silent']
    if len(y) > time_length:
        y = y[0:0+time_length]
    else:
        y = np.pad(y, (0,time_length-len(y)))
    return y

def wav2featuresflow(y, sr):
    y_without_silent = remove_silent(y,sr)
    melspectrogram = np.rot90(norm_mel(y, sr))
    return melspectrogram

def preprocessing_X(wav_dir):
    files = os.listdir(wav_dir)
    files.sort() #正確排序很重要!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    X = torch.FloatTensor([])
    for f in tqdm(files):
        try:
            samples, sample_rate = malaya_speech.load(wav_dir+f)
            melspectrogram = wav2featuresflow(samples,sr=sample_rate)
            melspectrogram = melspectrogram.reshape(-1,80,157).copy()
            melspectrogram = torch.from_numpy(melspectrogram)
            X = torch.cat([X,torch.unsqueeze(melspectrogram,0)],dim=0)
            gc.collect()
        except:
            print(f)
    print('X shape:',X.shape)
    return X

In [8]:
os.listdir()

['Electricssound',
 'dogbarking',
 'expansion_data.ipynb',
 'GlassBreakingsound',
 'download-from-youtube.sh',
 'dogcrying',
 'Vaccumsound',
 'doghowling',
 'dishessound',
 'Catsound',
 '.ipynb_checkpoints']

In [9]:
dirs = os.listdir()
dirs.sort

<function list.sort(*, key=None, reverse=False)>

In [10]:
dirs

['Electricssound',
 'dogbarking',
 'expansion_data.ipynb',
 'GlassBreakingsound',
 'download-from-youtube.sh',
 'dogcrying',
 'Vaccumsound',
 'doghowling',
 'dishessound',
 'Catsound',
 '.ipynb_checkpoints']

In [11]:
dirs = ['Electricssound',
        'dogbarking',
        'GlassBreakingsound',
        'dogcrying',
        'Vaccumsound',
        'doghowling',
        'dishessound',
        'Catsound']

In [ ]:
data = {}
for class_name in tqdm(dirs):
    wav_dir = class_name+'/'
    print(class_name)
    data[class_name] = preprocessing_X(wav_dir)

  0%|          | 0/20 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=480
  n_fft, y.shape[-1]


Electricssound


/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=320
  n_fft, y.shape[-1]

  0%|          | 0/260 [00:00<?, ?it/s]

X shape: torch.Size([20, 1, 80, 157])
dogbarking



  6%|▌         | 15/260 [01:47<26:14,  6.43s/it]

110.wav



  6%|▌         | 16/260 [01:47<18:31,  4.55s/it]

111.wav


In [ ]:
joblib.dump(data,'expansion_data.pkl')